In [10]:
import pandas as pd
import numpy as np
import datetime


In [42]:
data = pd.read_csv('../data/data.csv').drop(columns={'Currency', 'Type', 'Labels', 'Wallet'}).rename(columns={
    'Date':'date_utc',
    'Category name':'category'})

data.columns= data.columns.str.lower()
data['trx_date'] = pd.to_datetime(data.date_utc, format="%Y-%m-%dT%H:%M:%S"
                                 ).dt.tz_convert('Asia/Jakarta').dt.date

data['trx_month'] = pd.to_datetime(data.trx_date, format="%Y-%m-%d"
                                  ).dt.to_period('M').apply(lambda r:r.start_time)

data['amount']=-data['amount']
data = data[data.amount>0][['trx_date', 'trx_month', 'category', 'amount', 'note', 'author']]
print(data.shape)
data.head()

(2060, 6)


,trx_date,trx_month,category,amount,note,author
0,2021-02-11,2021-02-01,Food & Drink,79000.0,Breakfast Mie Asoy,Leo
1,2021-02-11,2021-02-01,Gifts,21000.0,Gosend pie susu nina & melissa,Sonia P
2,2021-02-11,2021-02-01,Transport,1100000.0,Servis Mobil,Leo
3,2021-02-11,2021-02-01,Food & Drink,81000.0,Lunch waroenk kito,Sonia P
4,2021-02-12,2021-02-01,Groceries,90000.0,Hypermart - BBQ sinchia,Leo


In [28]:
df = data[(data.trx_month>='2022-01-01') & (data.trx_month<='2022-12-01')].reset_index(drop=True)
df=df.reset_index().rename(columns={'index':'id'})
df['id']+=1
df.head()

,id,trx_date,trx_month,category,amount,note,author
0,1,2022-01-01,2022-01-01,Bills & Fees,984993.0,Maintenance Belmont,Leo
1,2,2022-01-01,2022-01-01,Nami,218300.0,"Tissue, tissue basah, sleek",Sonia P
2,3,2022-01-01,2022-01-01,Food & Drink,70000.0,Mamat,Leo
3,4,2022-01-01,2022-01-01,Shopping,74000.0,Bumbu tomyum,Leo
4,5,2022-01-02,2022-01-01,Food & Drink,72500.0,Mcd,Leo


In [50]:
df.sort_values('amount')

,id,trx_date,trx_month,category,amount,note,author
924,925,2022-10-01,2022-10-01,Transport,3500.0,Grab nina,Sonia P
1131,1132,2022-11-21,2022-11-01,Travel,9000.0,Roaming Leo,Leo
778,779,2022-08-28,2022-08-01,Food & Drink,12000.0,McD,Leo
890,891,2022-09-24,2022-09-01,Transport,13000.0,Parkir lippo,Leo
786,787,2022-08-29,2022-08-01,Food & Drink,14400.0,Suki,Leo
...,...,...,...,...,...,...,...
734,735,2022-08-13,2022-08-01,Travel,2724637.0,Puncak Pass Stay 2Nights,Leo
172,173,2022-02-23,2022-02-01,Healthcare,3000000.0,Allianz,Leo
64,65,2022-01-29,2022-01-01,Healthcare,3000000.0,Allianz,Leo
249,250,2022-03-16,2022-03-01,Food & Drink,3330000.0,Dapurfit sonia,Sonia P


In [39]:
category = df.groupby(['category']).agg(
    {'amount':'sum', 'id':'count', 'trx_date':'max'}
).sort_values('amount', ascending=False).reset_index().rename(columns={'id':'trx'})

category = category[category!='Maternity']
category['amount_per_trx'] = (category.amount/category.trx).astype(int)
category.head()

,category,amount,trx,trx_date,amount_per_trx
0,Nami,45670800.0,156,2022-12-28,292761
1,Food & Drink,45116012.0,486,2022-12-31,92831
2,Transport,20671027.0,73,2022-12-29,283164
3,Groceries,20419569.0,104,2022-12-31,196342
4,Healthcare,16540189.0,33,2022-12-24,501217


In [52]:
import plotly.express as px
fig = px.bar(category, x='category', y='amount')
fig.show()

In [138]:
category_m = df.groupby(['trx_month', 'category']).agg(
    {'amount':'sum', 'id':'count'}
).sort_values('amount', ascending=False).reset_index().rename(columns={'id':'trx'})

category_m = category_m[category_m!='Maternity'].sort_values(['category', 'trx_month'])
category_m['amount_per_trx'] = (category_m.amount/category_m.trx).astype(int)
category_m=category_m.dropna()
category_m.head()

,trx_month,category,amount,trx,amount_per_trx
72,2022-01-01,Bills & Fees,984993.0,1,984993
63,2022-02-01,Bills & Fees,1289000.0,2,644500
65,2022-03-01,Bills & Fees,1276642.0,1,1276642
100,2022-04-01,Bills & Fees,564000.0,3,188000
40,2022-05-01,Bills & Fees,2002500.0,2,1001250


In [136]:
df[pd.isnull(df).any(axis=1)]

,id,trx_date,trx_month,category,amount,note,author
32,33,2022-01-11,2022-01-01,Coffee,106500.0,NaN,Leo
93,94,2022-02-03,2022-02-01,Laundry,60000.0,NaN,Leo


In [139]:
category_list = category_m.category.unique()
len(category_list)

18

In [151]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

max_row = 6
max_col = 3
fig = make_subplots(rows=max_row, cols=max_col)
row, col = 1,1

for cat in category_list:
    temp = category_m[category_m.category==cat]
    fig.append_trace(go.Scatter(x=temp.trx_month.to_numpy(),
                                y=temp.amount.to_numpy()),
                     row=row, col=col)
    fig.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.2, showarrow=False,
                       text=cat, row=row, col=col)

    print(cat, row, col)
    if col<max_col:
        col+=1
    else:
        row+=1
        col=1
        
fig.update_layout(height=1000)

fig.show()



Bills & Fees 1 1
Books 1 2
Coffee 1 3
Drink 2 1
Entertainment 2 2
Food & Drink 2 3
Gadget 3 1
Gifts 3 2
Groceries 3 3
Healthcare 4 1
Home 4 2
Laundry 4 3
Nami 5 1
Other 5 2
Shopping 5 3
Snack 6 1
Transport 6 2
Travel 6 3
